### Lab10. Patients Physical Activities Prediction using Boosting

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.tree import DecisionTreeClassifier

### Step 1 [Understand Data]

In [2]:
df = pd.read_csv("human_activity_clipped.csv")

In [3]:
df.head()

,Unnamed: 0,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0,0.272284,-0.017143,-0.106584,-0.984438,-0.986949,-0.985428,-0.985560,-0.985928,-0.984612,...,-0.612020,-0.891765,-0.043298,-0.173359,0.581853,-0.028842,0.533818,0.999507,-0.183528,LAYING
1,1,0.295983,-0.015322,-0.112112,-0.949491,-0.973776,-0.970503,-0.949401,-0.972080,-0.971777,...,0.002383,-0.481007,-0.003165,-0.081509,0.100942,-0.027632,0.441570,-0.315805,-0.681485,LAYING
2,2,0.300053,-0.018087,-0.108184,-0.981943,-0.982872,-0.936259,-0.982640,-0.983396,-0.938447,...,-0.551846,-0.863125,-0.020589,0.017579,0.120518,-0.163567,0.441892,-0.817761,0.178551,LAYING
3,3,0.279295,-0.017524,-0.109860,-0.972667,-0.991136,-0.981223,-0.971652,-0.990865,-0.980863,...,-0.587825,-0.880670,0.518345,-0.019843,0.654278,-0.552475,0.528955,-0.492714,-0.511868,LAYING
4,4,0.265142,-0.069291,-0.027058,-0.884655,-0.754350,-0.798074,-0.900493,-0.749072,-0.787325,...,0.009777,-0.483548,-0.007532,-0.004246,-0.173846,0.348528,0.633505,-0.433927,-0.574650,LAYING


In [4]:
df.columns

Index(['Unnamed: 0', 'tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y',
       'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y',
       'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y',
       'tBodyAcc-mad()-Z',
       ...
       'fBodyBodyGyroJerkMag-skewness()', 'fBodyBodyGyroJerkMag-kurtosis()',
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'Activity'],
      dtype='object', length=563)

In [5]:
df.shape

(1500, 563)

In [6]:
df.dtypes

Unnamed: 0                                int64
tBodyAcc-mean()-X                       float64
tBodyAcc-mean()-Y                       float64
tBodyAcc-mean()-Z                       float64
tBodyAcc-std()-X                        float64
                                         ...   
angle(tBodyGyroJerkMean,gravityMean)    float64
angle(X,gravityMean)                    float64
angle(Y,gravityMean)                    float64
angle(Z,gravityMean)                    float64
Activity                                 object
Length: 563, dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Columns: 563 entries, Unnamed: 0 to Activity
dtypes: float64(561), int64(1), object(1)
memory usage: 6.4+ MB


In [8]:
df.value_counts()

Unnamed: 0  tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  tBodyAcc-mad()-Y  tBodyAcc-mad()-Z  tBodyAcc-max()-X  tBodyAcc-max()-Y  tBodyAcc-max()-Z  tBodyAcc-min()-X  tBodyAcc-min()-Y  tBodyAcc-min()-Z  tBodyAcc-sma()  tBodyAcc-energy()-X  tBodyAcc-energy()-Y  tBodyAcc-energy()-Z  tBodyAcc-iqr()-X  tBodyAcc-iqr()-Y  tBodyAcc-iqr()-Z  tBodyAcc-entropy()-X  tBodyAcc-entropy()-Y  tBodyAcc-entropy()-Z  tBodyAcc-arCoeff()-X,1  tBodyAcc-arCoeff()-X,2  tBodyAcc-arCoeff()-X,3  tBodyAcc-arCoeff()-X,4  tBodyAcc-arCoeff()-Y,1  tBodyAcc-arCoeff()-Y,2  tBodyAcc-arCoeff()-Y,3  tBodyAcc-arCoeff()-Y,4  tBodyAcc-arCoeff()-Z,1  tBodyAcc-arCoeff()-Z,2  tBodyAcc-arCoeff()-Z,3  tBodyAcc-arCoeff()-Z,4  tBodyAcc-correlation()-X,Y  tBodyAcc-correlation()-X,Z  tBodyAcc-correlation()-Y,Z  tGravityAcc-mean()-X  tGravityAcc-mean()-Y  tGravityAcc-mean()-Z  tGravityAcc-std()-X  tGravityAcc-std()-Y  tGravityAcc-std()-Z  tGravityAcc-mad

### Step3. [ Build GradientBoostingClassifier]

In [9]:
X=df.drop('Activity',axis=1)
y=df.Activity

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=22)

In [11]:
model = GradientBoostingClassifier(n_estimators=100,learning_rate=1.0,max_depth=1,random_state=0)

In [12]:
model.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [13]:
y_pred=model.predict(X_test)
y_pred

array(['LAYING', 'LAYING', 'LAYING', 'SITTING', 'SITTING', 'LAYING',
       'WALKING', 'WALKING', 'SITTING', 'WALKING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'SITTING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'SITTING', 'SITTING', 'WALKING',
       'WALKING', 'SITTING', 'WALKING', 'SITTING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'LAYING',
       'SITTING', 'LAYING', 'WALKING', 'SITTING', 'SITTING', 'SITTING',
       'WALKING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'WALKING', 'WALKING', 'WALKING', 'LAYING', 'SITTING', 'LAYING',
       'LAYING', 'WALKING', 'WALKING', 'WALKING', 'SITTING', 'WALKING',
       'WALKING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING',
       '

In [14]:
accuracy_score(y_test,y_pred)

1.0

In [15]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       154
     SITTING       1.00      1.00      1.00       148
     WALKING       1.00      1.00      1.00       148

    accuracy                           1.00       450
   macro avg       1.00      1.00      1.00       450
weighted avg       1.00      1.00      1.00       450



### Step4. [Find Best no. of trees and Best Learning Rate using Grid Search and Cross Validation]

In [16]:
Param_grid={'n_estimators':[50, 100, 200, 400],'learning_rate':[0.1, 0.01]}

In [17]:
classifier = GradientBoostingClassifier()

In [18]:
all_scores = cross_val_score(estimator=classifier,X=X_train,y=y_train,cv=5)

In [19]:
print(all_scores)

[1.        1.        1.        1.        0.9952381]


In [20]:
model2 = GridSearchCV(estimator=classifier,param_grid=Param_grid,cv=5,n_jobs=-1)

In [21]:
model2.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01],
                         'n_estimators': [50, 100, 200, 400]})

In [22]:
y_pred2=model2.predict(X_test)
y_pred2

array(['LAYING', 'LAYING', 'LAYING', 'SITTING', 'SITTING', 'LAYING',
       'WALKING', 'WALKING', 'SITTING', 'WALKING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'SITTING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'SITTING', 'SITTING', 'WALKING',
       'WALKING', 'SITTING', 'WALKING', 'SITTING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'LAYING',
       'SITTING', 'LAYING', 'WALKING', 'SITTING', 'SITTING', 'SITTING',
       'WALKING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'WALKING', 'WALKING', 'WALKING', 'LAYING', 'SITTING', 'LAYING',
       'LAYING', 'WALKING', 'WALKING', 'WALKING', 'SITTING', 'WALKING',
       'WALKING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING',
       '

In [23]:
accuracy_score(y_test,y_pred2)

1.0

In [24]:
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       154
     SITTING       1.00      1.00      1.00       148
     WALKING       1.00      1.00      1.00       148

    accuracy                           1.00       450
   macro avg       1.00      1.00      1.00       450
weighted avg       1.00      1.00      1.00       450



In [25]:
model2.best_estimator_

GradientBoostingClassifier(n_estimators=50)

### Step5. [Build AdaBoostClassifier]

In [26]:
base = DecisionTreeClassifier()

In [27]:
base2 = AdaBoostClassifier(base_estimator=base,random_state=0)

In [28]:
param_grid = {'n_estimators': [100, 150, 200], 'learning_rate': [0.01, 0.001]}

In [29]:
model3 = GridSearchCV(base2,param_grid,cv=10,n_jobs=-1)

In [30]:
model3.fit(X_train,y_train)

GridSearchCV(cv=10,
             estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),
                                          random_state=0),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.001],
                         'n_estimators': [100, 150, 200]})

In [31]:
y_pred3=model3.predict(X_test)
y_pred3

array(['LAYING', 'LAYING', 'LAYING', 'SITTING', 'SITTING', 'LAYING',
       'WALKING', 'WALKING', 'SITTING', 'WALKING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'SITTING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'SITTING', 'SITTING', 'WALKING',
       'WALKING', 'SITTING', 'WALKING', 'SITTING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'LAYING',
       'SITTING', 'LAYING', 'WALKING', 'SITTING', 'SITTING', 'SITTING',
       'WALKING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'WALKING', 'WALKING', 'WALKING', 'LAYING', 'SITTING', 'LAYING',
       'LAYING', 'WALKING', 'WALKING', 'WALKING', 'SITTING', 'WALKING',
       'WALKING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING',
       '

In [32]:
accuracy_score(y_test,y_pred3)

1.0

In [33]:
print(classification_report(y_test,y_pred3))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       154
     SITTING       1.00      1.00      1.00       148
     WALKING       1.00      1.00      1.00       148

    accuracy                           1.00       450
   macro avg       1.00      1.00      1.00       450
weighted avg       1.00      1.00      1.00       450



In [34]:
model3.best_estimator_

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), learning_rate=0.01,
                   n_estimators=100, random_state=0)

### Step6. [Build LogisticRegressionCV classifier]

In [35]:
model4 = LogisticRegressionCV(cv=4,Cs=5,penalty='l2')

In [36]:
model4.fit(X_train,y_train)

LogisticRegressionCV(Cs=5, cv=4)

In [37]:
y_pred4=model4.predict(X_test)
y_pred4

array(['LAYING', 'LAYING', 'LAYING', 'SITTING', 'SITTING', 'LAYING',
       'WALKING', 'WALKING', 'SITTING', 'WALKING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'SITTING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'SITTING', 'SITTING', 'WALKING',
       'WALKING', 'SITTING', 'WALKING', 'SITTING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'LAYING',
       'SITTING', 'LAYING', 'WALKING', 'SITTING', 'SITTING', 'SITTING',
       'WALKING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'WALKING', 'WALKING', 'WALKING', 'LAYING', 'SITTING', 'LAYING',
       'LAYING', 'WALKING', 'WALKING', 'WALKING', 'SITTING', 'WALKING',
       'WALKING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING',
       '

In [38]:
accuracy_score(y_test,y_pred4)

1.0

In [39]:
print(classification_report(y_test,y_pred4))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       154
     SITTING       1.00      1.00      1.00       148
     WALKING       1.00      1.00      1.00       148

    accuracy                           1.00       450
   macro avg       1.00      1.00      1.00       450
weighted avg       1.00      1.00      1.00       450



### Step 7 [ Build VotingClassifier]

In [40]:
model5=VotingClassifier(estimators=[('lr',model4),('gbc',base2)],voting='hard')

In [41]:
model5.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegressionCV(Cs=5, cv=4)),
                             ('gbc',
                              AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),
                                                 random_state=0))])

In [42]:
y_pred5=model5.predict(X_test)
y_pred5

array(['LAYING', 'LAYING', 'LAYING', 'SITTING', 'SITTING', 'LAYING',
       'WALKING', 'WALKING', 'SITTING', 'WALKING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'SITTING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'SITTING', 'SITTING', 'WALKING',
       'WALKING', 'SITTING', 'WALKING', 'SITTING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'LAYING',
       'SITTING', 'LAYING', 'WALKING', 'SITTING', 'SITTING', 'SITTING',
       'WALKING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'WALKING', 'WALKING', 'WALKING', 'LAYING', 'SITTING', 'LAYING',
       'LAYING', 'WALKING', 'WALKING', 'WALKING', 'SITTING', 'WALKING',
       'WALKING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING',
       '

In [43]:
accuracy_score(y_test,y_pred5)

1.0

In [44]:
print(classification_report(y_test,y_pred5))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       154
     SITTING       1.00      1.00      1.00       148
     WALKING       1.00      1.00      1.00       148

    accuracy                           1.00       450
   macro avg       1.00      1.00      1.00       450
weighted avg       1.00      1.00      1.00       450



### Step8. [ Interpret your results]

### GradientBoostingClassifier(n_estimators=50)

#####  GradientBoostingClassifier(n_estimators=50,learning_rate=1.0,max_depth=1,random_state=0)

In [45]:
model6 = GradientBoostingClassifier(n_estimators=50,learning_rate=1.0,max_depth=1,random_state=0)

In [46]:
model6.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, n_estimators=50,
                           random_state=0)

In [47]:
y_pred6=model6.predict(X_test)
y_pred6

array(['LAYING', 'LAYING', 'LAYING', 'SITTING', 'SITTING', 'LAYING',
       'WALKING', 'WALKING', 'SITTING', 'WALKING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'SITTING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'SITTING', 'SITTING', 'WALKING',
       'WALKING', 'SITTING', 'WALKING', 'SITTING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'LAYING',
       'SITTING', 'LAYING', 'WALKING', 'SITTING', 'SITTING', 'SITTING',
       'WALKING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'WALKING', 'WALKING', 'WALKING', 'LAYING', 'SITTING', 'LAYING',
       'LAYING', 'WALKING', 'WALKING', 'WALKING', 'SITTING', 'WALKING',
       'WALKING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING',
       '

In [48]:
accuracy_score(y_test,y_pred6)

1.0

In [49]:
print(classification_report(y_test,y_pred6))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       154
     SITTING       1.00      1.00      1.00       148
     WALKING       1.00      1.00      1.00       148

    accuracy                           1.00       450
   macro avg       1.00      1.00      1.00       450
weighted avg       1.00      1.00      1.00       450



### AdaBoostClassifier

##### AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), learning_rate=0.01, n_estimators=75, random_state=0)

In [50]:
base3 = AdaBoostClassifier(base_estimator=base,learning_rate=0.01,n_estimators=75,random_state=0)

In [51]:
model7 = GridSearchCV(base3,param_grid,cv=5,n_jobs=-1)

In [52]:
model7.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),
                                          learning_rate=0.01, n_estimators=75,
                                          random_state=0),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.001],
                         'n_estimators': [100, 150, 200]})

In [53]:
y_pred7=model7.predict(X_test)
y_pred7

array(['LAYING', 'LAYING', 'LAYING', 'SITTING', 'SITTING', 'LAYING',
       'WALKING', 'WALKING', 'SITTING', 'WALKING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'SITTING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'LAYING', 'SITTING', 'SITTING', 'SITTING', 'WALKING',
       'WALKING', 'SITTING', 'WALKING', 'SITTING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'LAYING', 'LAYING', 'LAYING',
       'SITTING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'LAYING', 'LAYING', 'SITTING', 'LAYING', 'WALKING', 'LAYING',
       'SITTING', 'LAYING', 'WALKING', 'SITTING', 'SITTING', 'SITTING',
       'WALKING', 'SITTING', 'SITTING', 'SITTING', 'WALKING', 'WALKING',
       'WALKING', 'WALKING', 'WALKING', 'LAYING', 'SITTING', 'LAYING',
       'LAYING', 'WALKING', 'WALKING', 'WALKING', 'SITTING', 'WALKING',
       'WALKING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING',
       '

In [54]:
accuracy_score(y_test,y_pred7)

1.0

In [55]:
print(classification_report(y_test,y_pred7))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       154
     SITTING       1.00      1.00      1.00       148
     WALKING       1.00      1.00      1.00       148

    accuracy                           1.00       450
   macro avg       1.00      1.00      1.00       450
weighted avg       1.00      1.00      1.00       450

